### Redshift catalog .fits

Andressa Wille

andressaw2@gmail.com

14/01/2026

* This is a tutorial on how to handle catalogs that are in FITS files.
In this example we are using a catalog of photometric redshifts for Abell 2744 galaxies: https://jwst-uncover.github.io/DR3.html#PhotometricCatalogs. Also see the description of the columns of the catalog.

Imports:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u

To read a fits file and see the catalog columns, we do:

In [2]:
hdul = fits.open('./redshift_catalog-uncover-DR3.fits')
hdul.info()

Filename: ./redshift_catalog-uncover-DR3.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     55   74020R x 18C   [K, D, D, K, D, K, J, J, I, D, D, D, D, D, D, D, D, D]   


In [3]:
catalog = Table.read('./redshift_catalog-uncover-DR3.fits', format='fits')
print(catalog.colnames)

['id', 'ra', 'dec', 'flag_kron', 'use_aper', 'use_phot', 'id_DR2', 'id_msa', 'id_alma', 'z_spec', 'z_ml', 'z_16', 'z_50', 'z_84', 'mu_num_16', 'mu_num_50', 'mu_num_84', 'mu_ml']


In [4]:
catalog 

id,ra,dec,flag_kron,use_aper,use_phot,id_DR2,id_msa,id_alma,z_spec,z_ml,z_16,z_50,z_84,mu_num_16,mu_num_50,mu_num_84,mu_ml
,deg,deg,,arcsec,,,,,,,,,,,,,
int64,float64,float64,int64,float64,int64,int32,int32,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,3.624497346714102,-30.467330291996568,1,0.32,0,--,--,--,--,--,--,--,--,--,--,--,--
2,3.6242204193456753,-30.467311501934542,1,0.32,0,--,--,--,--,--,--,--,--,--,--,--,--
3,3.611559372999214,-30.467324967684373,1,0.32,0,--,--,--,--,--,--,--,--,--,--,--,--
4,3.6105793754583444,-30.467312680054782,1,0.32,0,--,--,--,--,--,--,--,--,--,--,--,--
5,3.6290063501225447,-30.46732083715172,1,0.32,0,--,--,--,--,--,--,--,--,--,--,--,--
6,3.615578552874537,-30.46731371511246,1,0.32,0,--,--,--,--,--,--,--,--,--,--,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74015,3.5026111799495965,-30.290241994791376,0,0.32,0,61643,--,--,--,3.2357941483035315,0.2617378522659394,0.9851905189955223,3.151883164694618,1.0,1.0,1.0,1.0


##### Example 1: 
Let's say we are interested in a galaxy in Abell 2744 with known coordinates (RA:00:14:08.88, Dec:-30:21:06.56). We want to find information about its redshift in the catalog. We convert its coordinates to degrees and do a cross-matching:

In [5]:
coord = SkyCoord('00h14m08.88s -30d21m06.56s', frame='icrs')

print(f"RA (degree): {coord.ra.deg}")
print(f"Dec (degree): {coord.dec.deg}")

RA (degree): 3.537
Dec (degree): -30.351822222222225


In [6]:
cat_coords = SkyCoord(ra=catalog['ra'], dec=catalog['dec'])  

idx, sep2d, _ = coord.match_to_catalog_sky(cat_coords)

In [7]:
catalog[idx]

id,ra,dec,flag_kron,use_aper,use_phot,id_DR2,id_msa,id_alma,z_spec,z_ml,z_16,z_50,z_84,mu_num_16,mu_num_50,mu_num_84,mu_ml
,deg,deg,,arcsec,,,,,,,,,,,,,
int64,float64,float64,int64,float64,int64,int32,int32,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64
54240,3.5370654294601818,-30.351784337972102,0,1.4,1,42680,--,--,--,0.3093100264338136,0.3054634212422249,0.3190757159817336,0.3298208972266584,1.0,1.012626762862103,1.0243107618365646,1.001525741000431


We found it! We now know that this galaxy does not have a spectroscopic redshift (z_spec) in this catalog, but it does have a photometric redshift (z_ml: maximum-likelihood redshift) of ~0.3.

In [8]:
sep2d

<Angle [6.79936222e-05] deg>

##### Example 2: 
Now we want to find all galaxies in a certain redshift range (0.3 < z_spec < 0.315). We apply a filter and write a new catalog:

In [9]:
filtered_catalog = catalog[(catalog['z_spec'] >= 0.3) & (catalog['z_spec'] <= 0.315)]

filtered_catalog.write("gal_abell2744.fits", overwrite=True)

In [10]:
filtered_catalog

id,ra,dec,flag_kron,use_aper,use_phot,id_DR2,id_msa,id_alma,z_spec,z_ml,z_16,z_50,z_84,mu_num_16,mu_num_50,mu_num_84,mu_ml
,deg,deg,,arcsec,,,,,,,,,,,,,
int64,float64,float64,int64,float64,int64,int32,int32,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64
1511,3.5846093800925436,-30.463555136987505,0,1.4,1,--,--,--,0.303139,0.37249020756760787,0.004191035484477844,0.05146307246299174,0.3604608869527167,1.0,1.0,1.0508862860893082,1.0610881267053365
2104,3.5875916481966663,-30.46028969850352,0,1.4,1,--,--,--,0.303612,0.021806176477228656,0.018046239018215518,0.17026485630294957,0.3568376629695786,1.0,1.0,1.0426923946815352,1.0
5362,3.5785038316097633,-30.448083058752843,0,1.4,1,--,--,--,0.3035,0.24472823301946586,0.20131804867414188,0.27533959818197407,0.31358705890244815,1.0,1.0,1.0043504411026711,1.0
6196,3.5863183521665767,-30.447774855698878,0,1.4,1,--,--,--,0.3078,0.28285869419717946,0.24954224701729016,0.2849201209084494,0.3322377658295495,1.0,1.0,1.0174358200241607,1.0
7575,3.6120129873753974,-30.44556665017552,0,1.4,1,930,--,--,0.309058,0.2536835503800666,0.2594700194885707,0.295005733448306,0.3313435122506798,1.0,1.0,1.0135371142124958,1.0
7961,3.619296319926781,-30.440091425912826,0,1.4,1,1703,--,--,0.312158,0.257875172095841,0.23808378115613843,0.2703023198492159,0.3119674728841851,1.0,1.0,1.0040668882003554,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59689,3.5353761187439443,-30.342543322325653,0,1.4,1,--,--,--,0.314321,0.30798018244884695,0.25573854874690854,0.30710789909038916,0.36815944391511696,1.0,1.0,1.0506040685030427,1.0


We found 128 galaxies. Let's write them to a .reg file so we can find these galaxies in DS9:

In [12]:
catalog_ = Table.read("gal_abell2744.fits")

with open("gal_abell2744.reg", "w") as f:
    f.write("# Region file format: DS9 version 4.1\n")
    f.write("fk5\n")

    for ra, dec in zip(catalog_['ra'], catalog_['dec']):
        f.write(f"circle({ra},{dec},2\")\n") 